# Импорт датасетов

Через Google Drive

In [ ]:
news = pd.read_csv('/content/drive/MyDrive/news_processed.csv')
df = pd.read_csv('/content/drive/MyDrive/train.csv')

Напрямую

In [ ]:
news = pd.read_csv('/content/news_processed.csv')
df = pd.read_csv('/content/train.csv')

# Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
import seaborn as sns

# Построение эмбеддингов

In [ ]:
from transformers import pipeline
pipe = pipeline("feature-extraction", model="DeepPavlov/rubert-base-cased",)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

def get_embs(batch):
  encoded_input = tokenizer(batch, return_tensors='pt', padding="longest", truncation=True, max_length=512)

  # Generate embeddings
  with torch.no_grad():
      outputs = model(**encoded_input)

  # Get the last hidden state of the model, which represents the embeddings
  embeddings = outputs.last_hidden_state

  # Average the embeddings to get a single sentence embedding
  sentence_embedding = torch.mean(embeddings, dim=1).squeeze()
  return sentence_embedding

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_embeddings(text_series, batch_size=32):
    text_series = text_series.astype(str)
    embeddings_list = []
    for i in range(0, len(text_series), batch_size):
        batch = text_series[i:i+batch_size].to_list()
        embeddings = get_embs(batch)
        embeddings_list.append(embeddings)
    return torch.concat(embeddings_list, axis=0)

news_embeddings_list = [get_embeddings(news[col]) for col in ['title', 'content']]

In [ ]:
pd.DataFrame(news_embeddings_list[0]).to_csv('news_embedding_title.csv')
pd.DataFrame(news_embeddings_list[1]).to_csv('news_embedding_content.csv')

In [ ]:
title = news_embeddings_list[0]
content = news_embeddings_list[1]

# Предобработка данных

Feature Engineering

In [ ]:
df['datetime'] = pd.to_datetime(df['DATE']+' '+df['TIME'])

In [ ]:
df = df.set_index(['TICKER', 'datetime'])

In [ ]:
news['datetime'] = pd.to_datetime(news['datetime'])

In [ ]:
df['datetime'] = pd.to_datetime(df['DATE']+' '+df['TIME'])
df = df.drop(['TIME','DATE','PER'], axis=1)

Преобразования датасетов

In [ ]:
df_outer_join = pd.merge(news, content, on='Unnamed: 0', how='outer')

In [ ]:
news_with_embeddings = pd.merge(df_outer_join, titles, on='Unnamed: 0', how = 'outer')

In [ ]:
dataset = []
for i, row in news_with_embeddings.iterrows():
  for tick in ('SBER', ):
    try:

      date = row['datetime']
      date -= dt.timedelta(seconds = date.second)
      td = dt.timedelta(hours=1)
      delta = df.loc[(tick, pd.Timestamp(date))] - df.loc[(tick, pd.Timestamp(date - td))]
      el_to_append = {
          'title': row['title'],
          'tick': tick,
          'score': delta['CLOSE'],
          'content': row['content'],
          'datetime':row['datetime'],
          'open':df.loc[(tick, pd.Timestamp(date))]['OPEN'],
          'close':df.loc[(tick, pd.Timestamp(date+td))]['CLOSE'],
          'low': df.loc[(tick, pd.Timestamp(date))]['LOW'],
          'high': df.loc[(tick, pd.Timestamp(date))]['HIGH'],
          'vol': df.loc[(tick, pd.Timestamp(date))]['VOL']

      }
      for i in range(768):
        el_to_append[f'{i}_x'] = row[f'{i}_x']
        el_to_append[f'{i}_y'] = row[f'{i}_y']
      dataset.append(el_to_append)
    except:
      pass


In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df = df.drop(['content', 'title'], axis = 1)

Экспорт итогового датасета

In [ ]:
ready_df.to_csv('dataframe_no_text.csv', index = False)